In [17]:
import requests
import time
import re
import pandas as pd
import pprint
import pycrfsuite

import eli5
#import lime
#from lime.lime_text import LimeTextExplainer


from sklearn_crfsuite import CRF 
from sklearn.model_selection import cross_val_predict # train_test_split <- 쓰려면 쓰기
from sklearn_crfsuite .metrics import flat_classification_report # metrics: measure(Accuracy, score 등) 도구

from nltk.probability import FreqDist
from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import sent_to_xy
from bs4 import BeautifulSoup
from konlpy.tag import Kkma as kkm
from konlpy.tag import Okt
#from konlpy.tag import Mecab
#import Mecab
import tweepy


# score 정보 -> (원하는 것) 어떤 문장이 몇점인지를 예측 -> 이 값이 평균 이상이면 긍정, 평균 미만이면 부정
#            -> 긍/부정 으로만 학습하면 결과가 더 부정확할 수도. 
#            -> (현행) 평균 이상이면 100, 미만 0으로 분류해서 학습, 적용

# kkm 형태소 분석기에서 사전 조정해서, 칼퇴, 워라밸 같은 우리가 사용하는 단어로 수정하기.
# raw data : 맞춤법 교정.
# 조사 같은 연결어 제외하기. feature 추가(전전단어와 후후단어)
# CRF : N-gram ?????  kkm 사전을 형태소로 나눠놓고 하는 걸로. 

# 긍정/부정 리뷰 스크래핑+키워드 추출+사전으로 추가(feature) 
# 명사(를 비롯한 주요 단어)인 pos tag 만 뽑아서 feature에서 가중치 주기. 
# feature: pos tag에서 규칙 찾아서(eg. NNG+NNG) 여러 단어 연접하여 새롭게 긍/부정어 사전 돌리기 ??? <- 복잡. 
# 쪼개진 긍부정어 사전(만들어진 txt) 찾아보기.

# 조사 없애기? pos tag 포기. 띄어쓰기 교정 + 맞춤법 교정 + CRF ...? 
# feat -1, +1 < 최고로 나쁘다 -> 최고 로 나쁘 다 : 이런 식으로 CRF 특징이 무효화 됨.  

# CRF가 아니라, 다른 방식으로 해보는 게 어떤지? eg. review classification처럼 
# summarization + 빈도수 처리 + .... + N-gram 활용
# 시각화: eli5 

In [2]:
# 참고 사이트 https://yeo0.github.io/data/2018/09/24/5.-%EB%A1%9C%EA%B7%B8%EC%9D%B8%EC%9D%B4-%ED%95%84%EC%9A%94%ED%95%9C-%EC%82%AC%EC%9D%B4%ED%8A%B8%EC%97%90%EC%84%9C%EC%9D%98-%ED%81%AC%EB%A1%A4%EB%A7%81/
# 참고 사이트 2 https://hashcode.co.kr/questions/9084/%EC%9E%A1%ED%94%8C%EB%9E%98%EB%8B%9B-%EB%A1%9C%EA%B7%B8%EC%9D%B8-%ED%9B%84-%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91-%EB%AC%B8%EC%9D%98%EB%93%9C%EB%A6%BD%EB%8B%88%EB%8B%A4-%E3%85%9C%E3%85%9C

# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

reviews = []

In [3]:
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    page = random.sample(range(1, pages),30)
    reviews = {}
    for i in page: # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [4]:
#dict_pos=[]
#dict_neg=[]
#for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/pos_pol_word.txt", 'rt', encoding='UTF8'):
#    dict_pos.append(line.split('\n')[0])
    
#for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/neg_pol_word.txt", 'rt', encoding='UTF8'):
#    dict_neg.append(line.split('\n')[0])

In [5]:
# 각 산업군의 기업 리뷰 스크래핑
reviews1=dict(ind_reviews(1001))

In [6]:
reviews1

{'타 관공서와 비교했을 때 업무가 독립적인면이있어 1년마다 자리가 바뀌는 업무상 어려움과 업무량의 업다운에도 불구하고 내일만 잘하면되는면이있음 모든 법원이 다 마찬가지임': '86',
 '여유롭고 느긋한 분위기급여는 아쉽지만 워라벨 보장 가능': '86',
 '도서관 일을 하는데 별로 어려운일은 없고 서로 할일만 하면 터치 안함': '86',
 '존중받으며 일 할 수 있는 곳, 썩은 회사 다니다가 오면 갑자기 공무원 시험이 보고 싶어지고 그런다. 다만 진짜 공무원은 힘들어 보이는': '86',
 '체계가 있음 면접도 체계가 있어서 좋은 인상을 받음': '86',
 '해외에서 공관 근무하며 자부심을 가지며 일 할 수 있음. 해당 국가도시에서 급여가 좋은편.\r한국에서 4대보험이 들어가서 좋음.': '86',
 '제가 일했던 팀 분위기는 좋은 편이였고 업무 강도가 강하지 않으며 퇴근 시간 준수합니다.': '86',
 '교회 기반이여서 사람대우 해줍니다!\r이쪽업계가 사람 대우안해주는곳이 많다보니.....': '86',
 '주말 근로자로 근무. 최저임금. 달에 약 2번 휴무일 가짐. 점심시간 및 휴게시간 가짐.': '86',
 '직원간 분위기 좋고 안정적인 직장입니다': '86',
 '무난하게 다니기 좋다 크게 좋지도 크게 나쁘지도 않다': '64',
 '도시와 농촌이 적절하게 섞여있는 경찰서가 많이 있음.': '64',
 '관 이상급 능력차이가 크고 계약직과 정규직의 차별이 심함. 공공기관의 프로세스 경험할 수 있음.': '64',
 '성취에 큰 뜻이 없다면 무난히 다닐 수 있는 좋은 회사': '64',
 '도시개발공사로 넘어오면서 발전하려는 노력이 보이고 있음. 회식이 적고 칼퇴가 가능함 무엇보다 사내분위기가 좋다.': '64',
 '광주 헬게이트 구청 투톱. 시 전입시험이 다가오면 다들 탈출을 꿈꾼다.': '64',
 '청사 이전 후 환경이 더욱 좋아짐야근 강요 없고 사내 분위기도 나쁘지 않음': '64',
 '워라밸 좋음. 수평적인분위기 속 잘 적응할 수 

In [7]:
text1=list(reviews1.keys())
score1=list(reviews1.values())

In [8]:
sum=0
for i in range(len(score1)):
    sum+=int(score1[i])
avr=sum/len(score1)
print("score 평균값: ", round(avr,3))

score 평균값:  61.567


In [9]:
#############긍부정분석기 추가!!!!
#############
#############
#############


In [10]:
#전체 리뷰 추출

def all_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    reviews = {}
    for i in range(pages+1):
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [11]:
#전체 리뷰 및 별점 딕셔너리화
codes = ['1001', '900', '704', '1004']
reviews={}

for i in range(0,len(codes)):
    reviews.update(dict(all_reviews(codes[i])))

In [12]:
len(reviews)

2548

In [13]:
text_all=list(reviews.keys())
score_all=list(reviews.values())

In [14]:
#긍/부정 사전을 만들기 위한 전체 평균값 산출
sum=0
for i in range(len(score1)):
    sum+=int(score_all[i])
avr_all=sum/len(score_all)
print("score 평균값: ", round(avr_all,3))

score 평균값:  9.057


In [15]:
#공통/ 업종별 불용어(자주 나오지만 문서 전체에 관련되어 있어 성능 체크에 의미가 없는 단어들) 선언
stopwords=["직장","회사","기업","업무","사람","직원","있음"]
stopwords1=["공공기관","공기업","공공","정부","부처","공무원","기관"]
stopwords2=["은행","금융","금융권","금융업","운용사","보험사","지점"]
stopwords3=["개발자","소프트웨어","IT"]
stopwords4=["세무","회계","회계사","세무사","지점","법인"]

In [18]:
#긍/부정 리뷰별 빈출단어 추출 및 사전화
#검색 성능을 높이기 위해 문장을 제일 잘게 쪼개는 kkma 사용

kkm=kkm()

text_kkm=[]

#형태소 분석 및 문장별 점수 매칭

for i in range(len(text1)):
    text_kkm.append(kkm.nouns(text_all[i]))
    text_kkm[i].append(score_all[i])
    
print(text_kkm)

[['대체적', '근무', '환경', '부서', '업무량', '시', '야근', '필요', '경우', '존재', '100'], ['정년', '보장', '도서', '도서대출반납', '대출', '반납', '다양', '프로그램', '운영', '운영중', '중', '100'], ['대한', '대한민국', '민국', '식량', '식량산업', '산업', '최전선', '분투', '해군', '소수', '소수정예', '정예', '종자', '종자관리', '관리', '전문', '전문기관', '기관', '소속', '소속직원', '직원', '대한민국전체', '전체', '200', '200명', '명', '96'], ['수평적', '회사', '회사분위기', '분위기', '상사', '눈치', '안', '안보고', '보고', '휴가', '내가', '일', '외국인', '사람', '저녁', '삶', '가능', '직장', '96'], ['민원인', '업무', '업무강도', '강도', '조사관', '친절', '94'], ['세종', '세종시', '시', '활동', '기업', '기업지원기관', '지원', '기관', '신생', '신생기관', '94'], ['도청', '근무', '충청남', '충청남도', '도', '총괄적', '업무', '수행', '예하', '시', '시군의', '군의', '정책', '그림', '곳', '94'], ['서울', '도심', '국제기구', '국제', '국제현안', '현안', '수', '관련', '커리어', '있음', '94'], ['자유', '분위기', '보수적', '공공', '공공기관', '기관', '생각', '때', '느낌', '유사', '94'], ['노동법', '곳', '직원', '전반적', '함', '악성', '민', '민원인', '원인', '상대', '가나', '있음', '94'], ['워라벨', '연봉', '세금', '해외', '지역', '근무', '점', '단점', '힘', '92'], ['근무', '연가', '보', '보수가', '수가', '일', '만족

In [19]:
pos_kkm=[]
morphs_kkm=[]
dic_stopwords=stopwords1+stopwords2+stopwords3+stopwords4+stopwords
#연관이 낮아 보이는 형태소와 1글자 음절, 불용어를 제외한 체언, 용언, 관형사, 부사만 뽑아내기

for i in range(len(text_all)):
    pos_kkm.append(kkm.pos(text_all[i]))

dic_kkm=[]

for i in range(len(pos_kkm)):
    dic_kkm.append([])
    for j in range(len(pos_kkm[i])):
        if pos_kkm[i][j][0] not in dic_stopwords:
            if pos_kkm[i][j][1].startswith('N') or pos_kkm[i][j][1].startswith('V')or pos_kkm[i][j][1].startswith('M'):
                dic_kkm[i].append(pos_kkm[i][j][0])
    dic_kkm[i].append(int(score_all[i]))


In [20]:
dic_kkm

[['대체적',
  '근무',
  '환경',
  '좋',
  '부서',
  '따르',
  '다르',
  '업무량',
  '몰리',
  '시',
  '야근',
  '필요',
  '경우',
  '존재',
  100],
 ['정년',
  '확실히',
  '보장',
  '도서',
  '대출',
  '반납',
  '아니',
  '다양',
  '프로그램',
  '운영',
  '중',
  '이',
  '좋',
  100],
 ['대한민국',
  '식량',
  '산업',
  '최전선',
  '분투',
  '해군',
  '같',
  '소수',
  '정예',
  '종자',
  '관리',
  '전문',
  '소속',
  '대한민국',
  '전체',
  '200',
  '명',
  '없',
  96],
 ['수평적',
  '이',
  '분위기',
  '좋',
  '상사',
  '눈치',
  '많이',
  '안',
  '보고',
  '휴가',
  '쓰',
  '내가',
  '하',
  '일',
  '열심히',
  '하',
  '되',
  '외국인',
  '일하',
  '것',
  '좋아하',
  '즐기',
  '저녁',
  '있',
  '삶',
  '가능',
  96],
 ['민원인', '많', '강도', '높', '조사관', '친절', 94],
 ['세종', '시', '활동', '좋', '지원', '이', '신생', '이', 94],
 ['도청',
  '근무',
  '충청남도',
  '관하',
  '총괄적',
  '이',
  '수행',
  '되',
  '예하',
  '시',
  '군의',
  '정책',
  '크',
  '그림',
  '그리',
  '곳',
  94],
 ['서울',
  '도심',
  '있',
  '국제기구',
  '국제',
  '현안',
  '다루',
  '수',
  '있',
  '관련',
  '커리어',
  '쌓',
  '수',
  94],
 ['자유', '분위기', '보수적', '이', '않', '상당히', '생각', '때', '느낌', '가장', '유사', 9

In [72]:
# 긍/부정 빈도수 체크하여 상위 50개를 사전으로 저장

dic_text_pos=[]
dic_text_neg=[]  

for i in range(len(dic_kkm)):
    for j in range(0, len(dic_kkm[i])):
        if type(dic_kkm[i][j]) != int:
            if len(dic_kkm[i][j])>1:
                if float(dic_kkm[i][ -1]) > avr:
                    dic_text_pos.append(dic_kkm[i][j])
                else:
                    dic_text_neg.append(dic_kkm[i][j])

fdist1=FreqDist(dic_text_pos)
dic_text_pos=fdist1.most_common(100)

fdist1=FreqDist(dic_text_neg)
dic_text_neg=fdist1.most_common(100)

In [73]:
dic_text_pos1=[]
dic_text_neg1=[]

for i in range(len(dic_text_pos)):
    dic_text_pos1.append(dic_text_pos[i][0])
    
for i in range(len(dic_text_neg)):
    dic_text_neg1.append(dic_text_neg[i][0])

print(dic_text_pos1)
print(dic_text_neg1)

['일하', '분위기', '근무', '다니', '경험', '다양', '문화', '배우', '성장', '복지', '환경', '생각', '가지', '편하', '연봉', '추천', '안정적', '대하', '조직', '가능', '계약', '최고', '급여', '자유', '발전', '아니', '많이', '강도', '나쁘', '워라밸', '시간', '괜찮', '보장', '대우', '업계', '매우', '힘들', '수평적', '워라벨', '본인', '여러', '노력', '투자', '사업', '따르', '경력', '커리어', '비하', '기회', '생활', '함께', '자기', '개발', '부서', '하지만', '너무', '장점', '사회', '정도', '위하', '없이', '다만', '그냥', '자부심', '이상', '다른', '보수적', '체계', '보이', '능력', '하나', '느끼', '성과', '편안', '인턴', '운용', '스타트', '외국계', '다르', '관련', '전반적', '신입', '부족', '차이', '안정성', '만큼', '개인', '수준', '빠르', '사내', '눈치', '지원', '만족', '모르', '잘하', '때문', '월급', '정규직', '자산', '야근']
['다니', '분위기', '일하', '배우', '생각', '문화', '추천', '다양', '경험', '복지', '아니', '연봉', '많이', '힘들', '너무', '경력', '근무', '대하', '신입', '괜찮', '급여', '편하', '체계', '성장', '발전', '그냥', '환경', '자유', '대표', '사무실', '가지', '안정적', '매우', '나쁘', '따르', '가능', '다르', '위하', '시간', '개인', '못하', '사업', '보수적', '하지만', '계약', '바뀌', '업계', '모르', '없이', '이상', '입사', '야근', '보험', '운영', '오래', '대우', '조직', '본인', '자기', '별로', '개발', '여러', '가족', '

In [74]:
import site; site.getsitepackages()


['C:\\ProgramData\\Anaconda3\\envs\\tm',
 'C:\\ProgramData\\Anaconda3\\envs\\tm\\lib\\site-packages']

In [75]:
sent_num=0
text1_sent=[]
#mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
for i in range(len(text1)):
    text1_pos=kkm.pos(text1[i])
    sent_num+=1
    for t in text1_pos:
        n=list(t)
        
        sent_number="Sentence: {}".format(sent_num)
        n.insert(0, sent_number)
        '''
        if n[1] in dict_pos: 
            pos_dict=True
        else: 
            pos_dict=False
        n.insert(3, pos_dict)
        if n[1] in dict_neg: 
            neg_dict=True
        else: 
            neg_dict=False
        n.insert(4, neg_dict)
        '''
        if float(score1[(sent_num-1)]) > avr:
            n.insert(3, '100')
        else: 
            n.insert(3, '0')
        
        #n.insert(5, score1[(sent_num-1)])
        
        text1_sent.append(n)
        

pprint.pprint(text1_sent)

[['Sentence: 1', '타', 'VV', '100'],
 ['Sentence: 1', '아', 'ECS', '100'],
 ['Sentence: 1', '관공서', 'NNG', '100'],
 ['Sentence: 1', '와', 'JKM', '100'],
 ['Sentence: 1', '비교', 'NNG', '100'],
 ['Sentence: 1', '하', 'XSV', '100'],
 ['Sentence: 1', '었', 'EPT', '100'],
 ['Sentence: 1', '을', 'ETD', '100'],
 ['Sentence: 1', '때', 'NNG', '100'],
 ['Sentence: 1', '업무', 'NNG', '100'],
 ['Sentence: 1', '가', 'JKS', '100'],
 ['Sentence: 1', '독립적', 'NNG', '100'],
 ['Sentence: 1', '이', 'VCP', '100'],
 ['Sentence: 1', 'ㄴ', 'ETD', '100'],
 ['Sentence: 1', '면', 'NNG', '100'],
 ['Sentence: 1', '이', 'JKS', '100'],
 ['Sentence: 1', '있', 'VV', '100'],
 ['Sentence: 1', '어', 'ECD', '100'],
 ['Sentence: 1', '1', 'NR', '100'],
 ['Sentence: 1', '년', 'NNM', '100'],
 ['Sentence: 1', '마다', 'JX', '100'],
 ['Sentence: 1', '자리', 'NNG', '100'],
 ['Sentence: 1', '가', 'JKS', '100'],
 ['Sentence: 1', '바뀌', 'VV', '100'],
 ['Sentence: 1', '는', 'ETD', '100'],
 ['Sentence: 1', '업무상', 'NNG', '100'],
 ['Sentence: 1', '어려움', 'NNG', '

In [76]:
text1_sent

[['Sentence: 1', '타', 'VV', '100'],
 ['Sentence: 1', '아', 'ECS', '100'],
 ['Sentence: 1', '관공서', 'NNG', '100'],
 ['Sentence: 1', '와', 'JKM', '100'],
 ['Sentence: 1', '비교', 'NNG', '100'],
 ['Sentence: 1', '하', 'XSV', '100'],
 ['Sentence: 1', '었', 'EPT', '100'],
 ['Sentence: 1', '을', 'ETD', '100'],
 ['Sentence: 1', '때', 'NNG', '100'],
 ['Sentence: 1', '업무', 'NNG', '100'],
 ['Sentence: 1', '가', 'JKS', '100'],
 ['Sentence: 1', '독립적', 'NNG', '100'],
 ['Sentence: 1', '이', 'VCP', '100'],
 ['Sentence: 1', 'ㄴ', 'ETD', '100'],
 ['Sentence: 1', '면', 'NNG', '100'],
 ['Sentence: 1', '이', 'JKS', '100'],
 ['Sentence: 1', '있', 'VV', '100'],
 ['Sentence: 1', '어', 'ECD', '100'],
 ['Sentence: 1', '1', 'NR', '100'],
 ['Sentence: 1', '년', 'NNM', '100'],
 ['Sentence: 1', '마다', 'JX', '100'],
 ['Sentence: 1', '자리', 'NNG', '100'],
 ['Sentence: 1', '가', 'JKS', '100'],
 ['Sentence: 1', '바뀌', 'VV', '100'],
 ['Sentence: 1', '는', 'ETD', '100'],
 ['Sentence: 1', '업무상', 'NNG', '100'],
 ['Sentence: 1', '어려움', 'NNG', '

In [77]:
#text2_del_SW = []
#for i in range(len(text2_sent)):
#    for t in text1_sent : 
#        if ( (t[2] != 'ETD') | (t[2] !='SF') | (t[2] !='SP') | (t[2] !='XSV') 
#            | (t[2][:2] !='JK')   ):
#            text2_del_SW.append(t)

In [78]:
#text2_del_SW

In [79]:
#text2_del_SW = []
#for i in range(len(text2_sent)):
#    for t in text2_sent : 
#        if t[1] != 'ETD':
#            text2_del_SW.append()

In [80]:
#dict_pos

In [81]:
#dict_neg

In [82]:
data = pd.DataFrame(text1_sent) # data를 pandas로 변환

In [83]:
data.columns = ['Sentence #', 'Word', 'POS', 'score']

In [84]:
data.head(10)

,Sentence #,Word,POS,score
0,Sentence: 1,타,VV,100
1,Sentence: 1,아,ECS,100
2,Sentence: 1,관공서,NNG,100
3,Sentence: 1,와,JKM,100
4,Sentence: 1,비교,NNG,100
5,Sentence: 1,하,XSV,100
6,Sentence: 1,었,EPT,100
7,Sentence: 1,을,ETD,100
8,Sentence: 1,때,NNG,100
9,Sentence: 1,업무,NNG,100


In [85]:
data.tail(10)

,Sentence #,Word,POS,score
7223,Sentence: 300,넘,VV,100
7224,Sentence: 300,게,ECD,100
7225,Sentence: 300,보상,NNG,100
7226,Sentence: 300,받,VV,100
7227,Sentence: 300,을,ETD,100
7228,Sentence: 300,수,NNB,100
7229,Sentence: 300,있,VV,100
7230,Sentence: 300,는,ETD,100
7231,Sentence: 300,직장,NNG,100
7232,Sentence: 300,.,SF,100


In [86]:
data.isnull().sum() # 비어있는 값 확인 결과: 없음

Sentence #    0
Word          0
POS           0
score         0
dtype: int64

In [87]:
words = list(set(data["Word"].values))
n_words = len(words) # 고유한 단어들이 1784개(문장부호 포함)
n_words

1459

In [88]:
# 데이터에서 문장을 원하는 형태로 포매팅하여 추출하는 클래스
# Word, POS -> data를 여러 함수로 씌워서 봄. 
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        # for문 사용법 보기. lambda: 일종의 함수(적용하는), data에 다음과 같은 기능 적용
        agg_func = lambda s: [(w, p, s) for w, p, s in zip(s["Word"].values.tolist(),
                                                                     s["POS"].values.tolist(),
                                                                     s["score"].values.tolist())]
        # sentence 단위로 grouped
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        # 그룹화된 데이터를 다시 sentences로 저장
        self.sentences = [s for s in self.grouped]
        print(self.grouped)
        self.sentences
    
    # get_next: 다음 문장 처리
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


In [89]:
# 전체 문장 추출
getter = SentenceGetter(data)

Sentence #
Sentence: 1      [(타, VV, 100), (아, ECS, 100), (관공서, NNG, 100),...
Sentence: 10     [(직원, NNG, 100), (간, NNG, 100), (분위기, NNG, 100...
Sentence: 100    [(사업, NNG, 0), (은, JX, 0), (점점, MAG, 0), (개선, ...
Sentence: 101    [(사고, NNG, 0), (만, JX, 0), (안, MAG, 0), (치, VV...
Sentence: 102    [(수성구, NNP, 0), (범어, NNG, 0), (역, NNG, 0), (에,...
                                       ...                        
Sentence: 95     [(주변, NNG, 0), (에, JKM, 0), (학교, NNG, 0), (도, ...
Sentence: 96     [(서울, NNG, 0), (여대, NNG, 0), (위탁, NNG, 0), (으로...
Sentence: 97     [(초보자, NNG, 0), (가, JKS, 0), (근무, NNG, 0), (하,...
Sentence: 98     [(정규직, NNG, 0), (에게, JKM, 0), (는, JX, 0), (회사,...
Sentence: 99     [(타, VV, 0), (아, ECS, 0), (여성, NNG, 0), (인력, N...
Length: 300, dtype: object


In [90]:
# 문장 하나
sent = getter.get_next()

In [91]:
# 추출한 문장 확인(적용 결과)
print(sent)

[('타', 'VV', '100'), ('아', 'ECS', '100'), ('관공서', 'NNG', '100'), ('와', 'JKM', '100'), ('비교', 'NNG', '100'), ('하', 'XSV', '100'), ('었', 'EPT', '100'), ('을', 'ETD', '100'), ('때', 'NNG', '100'), ('업무', 'NNG', '100'), ('가', 'JKS', '100'), ('독립적', 'NNG', '100'), ('이', 'VCP', '100'), ('ㄴ', 'ETD', '100'), ('면', 'NNG', '100'), ('이', 'JKS', '100'), ('있', 'VV', '100'), ('어', 'ECD', '100'), ('1', 'NR', '100'), ('년', 'NNM', '100'), ('마다', 'JX', '100'), ('자리', 'NNG', '100'), ('가', 'JKS', '100'), ('바뀌', 'VV', '100'), ('는', 'ETD', '100'), ('업무상', 'NNG', '100'), ('어려움', 'NNG', '100'), ('과', 'JKM', '100'), ('업무량', 'NNG', '100'), ('의', 'JKG', '100'), ('업다운', 'NNG', '100'), ('에', 'JKM', '100'), ('도', 'JX', '100'), ('불구', 'XR', '100'), ('하', 'XSA', '100'), ('고', 'ECE', '100'), ('내일', 'NNG', '100'), ('만', 'JX', '100'), ('잘하', 'VV', '100'), ('면', 'ECE', '100'), ('되', 'VV', '100'), ('는', 'ETD', '100'), ('면', 'NNG', '100'), ('이', 'JKS', '100'), ('있음', 'NNG', '100'), ('모든', 'NNP', '100'), ('법원', 'NNG', '100'),

In [92]:
# 전체 문장
sentences = getter.sentences

In [93]:
print(len(sentences)) # 개수 확인

# 문장 하나 확인
print(sentences[58])
print(sentences[:2]) # list의 list 형태로 여러 개 표시. 


# 각각 word, pos, tag, pare로 만들어서 결과 확인

300
[('서울', 'NNG', '100'), ('이', 'VCP', '100'), ('라는', 'ETD', '100'), ('대도시', 'NNG', '100'), ('안', 'NNG', '100'), ('에', 'JKM', '100'), ('그것', 'NP', '100'), ('도', 'JX', '100'), ('숲', 'NNG', '100'), ('속', 'NNG', '100'), ('안', 'NNG', '100'), ('에서', 'JKM', '100'), ('근무', 'NNG', '100'), ('환경', 'NNG', '100'), ('이', 'JKS', '100'), ('조성', 'NNG', '100'), ('되', 'XSV', '100'), ('어', 'ECD', '100'), ('있', 'VXV', '100'), ('다는', 'ETD', '100'), ('아주', 'MAG', '100'), ('좋', 'VA', '100'), ('은', 'ETD', '100'), ('근무', 'NNG', '100'), ('조건', 'NNG', '100'), ('을', 'JKO', '100'), ('가지', 'VV', '100'), ('고', 'ECE', '100'), ('있', 'VXV', '100'), ('다', 'EFN', '100'), ('.', 'SF', '100')]
[[('타', 'VV', '100'), ('아', 'ECS', '100'), ('관공서', 'NNG', '100'), ('와', 'JKM', '100'), ('비교', 'NNG', '100'), ('하', 'XSV', '100'), ('었', 'EPT', '100'), ('을', 'ETD', '100'), ('때', 'NNG', '100'), ('업무', 'NNG', '100'), ('가', 'JKS', '100'), ('독립적', 'NNG', '100'), ('이', 'VCP', '100'), ('ㄴ', 'ETD', '100'), ('면', 'NNG', '100'), ('이', 'JKS', 

In [94]:
# 단어를 features화
# "feat의 이름" : "value"
# 하나의 단어에서 다양한 feature(단어, POS tag, lables 등) 확인을 위해 문장 전체를 넣어줌. 

def word2features(sent, i):
    word = sent[i][0]   # index0의 단어를 가져옴
    postag = sent[i][1] # index1의 단어를 가져옴
    score = sent[i][2]

# postag 유지
# postag2 = kkma.pos(word)
# 
    features = {
        'bias': 1.0,
        'postag': postag,
        'postag[:2]': postag[:2],
        'dic_text_pos' : True if word in dic_text_pos1 else False,
        'dic_text_neg' : True if word in dic_text_neg1 else False,
#        'dict_pos' : True if word in dict_pos else False,
#        'dict_neg' : True if word in dict_neg else False,
        'word.len' : len(word)
        
# 특징!!!! 
# 
#'dict_time': word.lower() in dict_time # -> true or false로
# character 개수 
# 'word.len' : len(word)
    }
    
    # 직전의 단어에 대해서도 똑같이 적용
    if i > 0:
        word1 = sent[i-1][0] # i-1: 직전의 단어
        postag1 = sent[i-1][1]
        # update: 기존에서 추가! (하여 새로운 dictionary를 만드는 결과)
        features.update({
            '-1:postag': postag,
            'dic_text_pos' : True if word in dic_text_pos1 else False,
            'dic_text_neg' : True if word in dic_text_neg1 else False,
#            'dict_pos' : True if word in dict_pos else False,
#            'dict_neg' : True if word in dict_neg else False,
            '-1:word.len' : len(word)
        })
    #else:
    #    features['BOS'] = True # BOS: Beginning Of Sentence

    # 다음 단어에 대해서도 똑같이 처리
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:postag': postag,
            'dic_text_pos' : True if word in dic_text_pos1 else False,
            'dic_text_neg' : True if word in dic_text_neg1 else False,
#            'dict_pos' : True if word in dict_pos else False,
#            'dict_neg' : True if word in dict_neg else False,
            '+1:word.len' : len(word)
        })
    
    #else:
    #    features['EOS'] = True  # EOS: end of sentence
        
    # bi-그램에 대해서도 똑같이 처리
    
    return features

# sentence 전체에 적용하기 위한 함수
def sent2features(sent):
    for i in range(len(sent)):
        print(word2features(sent, i))
    return [word2features(sent, i) for i in range(len(sent))] # 0~i(index의 수)까지 적용 -> sentence에 적용
# == features = [word2features(sent, i) for i in range(len(sent))]
# return features

def sent2score(sent): # sent를 읽어서 for문을 돌림. label만 갖고 옴
    return [score for token, postag, score in sent]
# 지금은 3개가 tuple로 있음.

def sent2tokens(sent):
    return [token for token, postag, score in sent]

In [95]:
sent2score(sent)

['100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100']

In [96]:
print(sent2features(sent))

{'bias': 1.0, 'postag': 'VV', 'postag[:2]': 'VV', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '+1:postag': 'VV', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ECS', 'postag[:2]': 'EC', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '-1:postag': 'ECS', '-1:word.len': 1, '+1:postag': 'ECS', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 3, '-1:postag': 'NNG', '-1:word.len': 3, '+1:postag': 'NNG', '+1:word.len': 3}
{'bias': 1.0, 'postag': 'JKM', 'postag[:2]': 'JK', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '-1:postag': 'JKM', '-1:word.len': 1, '+1:postag': 'JKM', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'XSV', 'postag[:2]': 'XS', 'dic_text_pos': False, 'dic_text_neg': False, 'wo

In [97]:
X = [sent2features(s) for s in sentences] # sentence마다 feature로 바뀌고
y = [sent2score(s) for s in sentences] # sentence마다 score로 바뀜

{'bias': 1.0, 'postag': 'VV', 'postag[:2]': 'VV', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '+1:postag': 'VV', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ECS', 'postag[:2]': 'EC', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '-1:postag': 'ECS', '-1:word.len': 1, '+1:postag': 'ECS', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 3, '-1:postag': 'NNG', '-1:word.len': 3, '+1:postag': 'NNG', '+1:word.len': 3}
{'bias': 1.0, 'postag': 'JKM', 'postag[:2]': 'JK', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '-1:postag': 'JKM', '-1:word.len': 1, '+1:postag': 'JKM', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'XSV', 'postag[:2]': 'XS', 'dic_text_pos': False, 'dic_text_neg': False, 'wo

In [98]:
y

[['100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100'],
 ['100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100'],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '

In [99]:
print(len(X))
print(len(y))
print(len(sentences))

300
300
300


In [100]:
#10번째 sentence의 정보 
print(len(X[10])) 
print(len(y[10]))
print(sentences[10])
print(y[10])
print(X[10])

16
16
[('전시회', 'NNG', '0'), ('나', 'JC', '0'), ('행사', 'NNG', '0'), ('기획', 'NNG', '0'), (',', 'SP', '0'), ('마이', 'NNG', '0'), ('슬', 'VA', '0'), ('사', 'ECD', '0'), ('업', 'NNG', '0'), ('에', 'JKM', '0'), ('관심', 'NNG', '0'), ('있', 'VV', '0'), ('는', 'ETD', '0'), ('사람', 'NNG', '0'), ('에게', 'JKM', '0'), ('추천', 'NNG', '0')]
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
[{'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 3, '+1:postag': 'NNG', '+1:word.len': 3}, {'bias': 1.0, 'postag': 'JC', 'postag[:2]': 'JC', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 1, '-1:postag': 'JC', '-1:word.len': 1, '+1:postag': 'JC', '+1:word.len': 1}, {'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': False, 'dic_text_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}, {'bias': 1.0, 'postag': 'NNG', 'postag[:2]': 'NN', 'dic_text_pos': False, 'd

In [101]:
# 문장 하나 확인
# 각 단어의 features가 dictionary로 표현되고 이를 요소로 하는 리스트
X[0]
# 문장은 단어로 이루어져 있고, 각 단어의 features 확인 가능
# 첫 번째 단어는 앞 단어에 대한 정보는 없고 BOS = true

[{'bias': 1.0,
  'postag': 'VV',
  'postag[:2]': 'VV',
  'dic_text_pos': False,
  'dic_text_neg': False,
  'word.len': 1,
  '+1:postag': 'VV',
  '+1:word.len': 1},
 {'bias': 1.0,
  'postag': 'ECS',
  'postag[:2]': 'EC',
  'dic_text_pos': False,
  'dic_text_neg': False,
  'word.len': 1,
  '-1:postag': 'ECS',
  '-1:word.len': 1,
  '+1:postag': 'ECS',
  '+1:word.len': 1},
 {'bias': 1.0,
  'postag': 'NNG',
  'postag[:2]': 'NN',
  'dic_text_pos': False,
  'dic_text_neg': False,
  'word.len': 3,
  '-1:postag': 'NNG',
  '-1:word.len': 3,
  '+1:postag': 'NNG',
  '+1:word.len': 3},
 {'bias': 1.0,
  'postag': 'JKM',
  'postag[:2]': 'JK',
  'dic_text_pos': False,
  'dic_text_neg': False,
  'word.len': 1,
  '-1:postag': 'JKM',
  '-1:word.len': 1,
  '+1:postag': 'JKM',
  '+1:word.len': 1},
 {'bias': 1.0,
  'postag': 'NNG',
  'postag[:2]': 'NN',
  'dic_text_pos': False,
  'dic_text_neg': False,
  'word.len': 2,
  '-1:postag': 'NNG',
  '-1:word.len': 2,
  '+1:postag': 'NNG',
  '+1:word.len': 2},
 {'b

In [102]:
print(len(X), len(X)*0.7) #train-test split (70%, 30%)
print(type(X))

300 210.0
<class 'list'>


In [103]:
crf = CRF(algorithm='lbfgs',
          c1=0.1, #c1의 가중치를 조절하여 feature의 영향력 조절
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [104]:

# Training
crf.fit(X[:300], y[:300])

C:\ProgramData\Anaconda3\envs\tm\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [105]:
# cross validation
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5) 

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [106]:
# prediction
y_test_pred = crf.predict(X[300:])

In [107]:
# report
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

# data + deep learning 적용 시 늘어날 가능성은 있음. 

              precision    recall  f1-score   support

           0       0.52      0.49      0.50      3560
         100       0.53      0.57      0.55      3673

    accuracy                           0.53      7233
   macro avg       0.53      0.53      0.53      7233
weighted avg       0.53      0.53      0.53      7233



C:\ProgramData\Anaconda3\envs\tm\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [108]:
# 태그 간의 transition(전이) probabilities, 태그 별 예측에 중요한 features
# eli5: weight visualization
eli5.show_weights(crf, top=30) # -> 중요한 것만 확인

In [70]:
# 시각화 하려고 했지만 실패한 흔적

explainer = LimeTextExplainer(class_names='CRF_NER')

NameError: name 'LimeTextExplainer' is not defined

In [71]:
exp1 = explainer.explain_instance(text1[0], pred, num_features=5)

NameError: name 'explainer' is not defined